# Workshop 6: Logistic Regression
## Data Science for Mathematicians

This workshop implements logistic regression from first principles, using the **Breast Cancer Wisconsin dataset** to classify tumors as malignant or benign. We move from the sigmoid function and binary cross-entropy loss to gradient descent for classification, then compare our from-scratch implementation with Scikit-learn.

**Key concepts covered:**
- The sigmoid function $\sigma(z) = 1/(1+e^{-z})$ and its inverse, the logit
- Binary cross-entropy (BCE) loss: derivation and information-theoretic interpretation
- Gradient descent for logistic regression: $\nabla J = \mathbf{X}^T(\mathbf{p} - \mathbf{y})$
- Convexity of the BCE loss and convergence guarantees
- Decision boundaries and classification thresholds
- Confusion matrix, precision, recall, F1-score, and the precision-recall trade-off
- Odds ratios and coefficient interpretation
- Feature selection via coefficient magnitude

## Section 0: Setup and Data Loading

***

## Section 1: The Sigmoid and Logit Functions

**Theory recap.**  The **sigmoid function** maps any real number to the interval $(0, 1)$:
$$
\sigma(z) = \frac{1}{1 + e^{-z}}.
$$
Its inverse is the **logit function**, which maps a probability $p \in (0, 1)$ back to the real line:
$$
\operatorname{logit}(p) = \log\frac{p}{1-p}.
$$
These two functions form a bijection between $\mathbb{R}$ and $(0,1)$, and are the foundation of logistic regression: we model $P(Y=1 \mid \mathbf{x}) = \sigma(\mathbf{x}^T\boldsymbol{\beta})$.

### Question 1.1: Implement the Sigmoid Function

Write a function `sigmoid(z)` that computes $\sigma(z) = 1/(1+e^{-z})$.  Then:
1. Plot $\sigma(z)$ over $z \in [-8, 8]$
2. Mark the point $\sigma(0) = 0.5$
3. Verify the symmetry property $\sigma(-z) = 1 - \sigma(z)$ numerically

### Question 1.2: Implement the Logit Function

Write a function `logit(p)` that computes $\operatorname{logit}(p) = \log(p/(1-p))$.  Verify the inverse relationship $\operatorname{logit}(\sigma(z)) = z$ and plot both functions on the same axes.

**Interpretation.** The sigmoid squashes any real-valued linear combination $\mathbf{x}^T\boldsymbol{\beta}$ into a valid probability. The logit is its inverse, mapping probabilities back to log-odds. Together they provide the link between the linear predictor and the probability of class membership.

***

## Section 2: Binary Cross-Entropy Loss

**Theory recap.**  Given predicted probabilities $p_i = \sigma(\mathbf{x}_i^T\boldsymbol{\beta})$ and true labels $y_i \in \{0, 1\}$, the **binary cross-entropy (BCE) loss** is
$$
J(\boldsymbol{\beta}) = -\frac{1}{n}\sum_{i=1}^n \bigl[y_i \log p_i + (1 - y_i)\log(1 - p_i)\bigr].
$$
This is the negative log-likelihood of the Bernoulli model. It penalizes confident wrong predictions exponentially: if $y_i = 1$ but $p_i \to 0$, then $-\log p_i \to \infty$.

### Question 2.1: Implement BCE Loss and Visualize the Penalty Structure

Write a function `bce_loss(y, p)` that computes the binary cross-entropy.  Then plot the per-sample loss $\ell(y, p)$ as a function of $p$ for both $y = 1$ and $y = 0$ on the same axes.

### Question 2.2: Baseline BCE Loss

Compute the BCE loss on the training data at $\boldsymbol{\beta} = \mathbf{0}$ (all predictions are $\sigma(0) = 0.5$). This serves as a **baseline**: any trained model should achieve lower loss than this uninformative prediction. Compare with a random $\boldsymbol{\beta}$.

**Interpretation.** At $\boldsymbol{\beta} = \mathbf{0}$, every sample is predicted with probability 0.5, giving a loss of $\log 2 \approx 0.6931$---the maximum entropy of a Bernoulli distribution. This is the worst-case "no information" baseline. The random $\boldsymbol{\beta}$ may do slightly better or worse by chance.

***

## Section 3: Gradient Descent for Logistic Regression

**Theory recap.**  The gradient of the BCE loss with respect to $\boldsymbol{\beta}$ is
$$
\nabla J(\boldsymbol{\beta}) = \frac{1}{n}\mathbf{X}^T(\mathbf{p} - \mathbf{y}),
$$
where $\mathbf{p} = \sigma(\mathbf{X}\boldsymbol{\beta})$ is the vector of predicted probabilities. The update rule is
$$
\boldsymbol{\beta}_{k+1} = \boldsymbol{\beta}_k - \eta\,\nabla J(\boldsymbol{\beta}_k).
$$
The BCE loss is **convex** in $\boldsymbol{\beta}$ (since the Hessian $\mathbf{H} = \frac{1}{n}\mathbf{X}^T\mathbf{D}\mathbf{X}$ with $\mathbf{D} = \operatorname{diag}(p_i(1-p_i))$ is PSD), so gradient descent converges to the global minimum.

### Question 3.1: Logistic Regression from Scratch (2 Features)

Implement gradient descent for logistic regression using only two features (`radius_mean` and `texture_mean`) for visualization. Track the loss history and plot the convergence curve.

### Question 3.2: Visualize the 2D Decision Boundary

The decision boundary is the set of points where $\sigma(\mathbf{x}^T\boldsymbol{\beta}) = 0.5$, i.e., $\mathbf{x}^T\boldsymbol{\beta} = 0$.  For the 2-feature model, this is a line:
$$
\beta_0 + \beta_1 x_1 + \beta_2 x_2 = 0 \quad\Longrightarrow\quad x_2 = -\frac{\beta_0 + \beta_1 x_1}{\beta_2}.
$$
Plot the training data and the decision boundary.

### Question 3.3: Full 30-Feature Model

Train logistic regression with all 30 features (standardized). Compare convergence speed with the 2-feature model and print the final coefficients.

**Interpretation.** The 30-feature model achieves a substantially lower BCE loss than the 2-feature model, showing that the additional features carry discriminative information. However, the 30-feature model converges more slowly per iteration due to the higher-dimensional parameter space and potentially worse conditioning.

***

## Section 4: Model Evaluation

**Theory recap.**  For a binary classifier with threshold $\tau$, we predict $\hat{y}_i = 1$ if $p_i \ge \tau$.  The **confusion matrix** counts true positives (TP), false positives (FP), true negatives (TN), and false negatives (FN).  From these we derive:
$$
\text{Accuracy} = \frac{\text{TP}+\text{TN}}{n}, \quad
\text{Precision} = \frac{\text{TP}}{\text{TP}+\text{FP}}, \quad
\text{Recall} = \frac{\text{TP}}{\text{TP}+\text{FN}}, \quad
F_1 = \frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}.
$$

### Question 4.1: Confusion Matrix and Metrics from Scratch

Using the full 30-feature model on the **test set**: compute predicted probabilities, apply threshold $\tau = 0.5$, build the confusion matrix from scratch, and compute accuracy, precision, recall, and F1-score.

### Question 4.2: ROC/AUC, $F_\beta$-Scores, and the Precision-Recall Trade-off

**(a)** Vary the classification threshold $\tau$ from 0 to 1.  For each $\tau$, compute the true positive rate (TPR = recall) and false positive rate (FPR) from scratch.  Plot the **ROC curve** (FPR vs TPR) and compute the **AUC** using the trapezoidal rule.

**(b)** The $F_\beta$-score generalises $F_1$ by weighting recall $\beta^2$ times as heavily as precision:
$$
F_\beta = (1 + \beta^2)\,\frac{\text{precision} \cdot \text{recall}}{\beta^2 \cdot \text{precision} + \text{recall}}.
$$
Compute $F_{0.5}$ (precision-focused), $F_1$ (balanced), and $F_2$ (recall-focused) as functions of $\tau$.  Plot all three on a single axes and mark the threshold that maximises each.

**(c)** In medical diagnosis, a **false negative** (missed cancer) is far more costly than a false positive (unnecessary biopsy).  Which $F_\beta$ variant is most appropriate, and what threshold does it suggest?  Compare this with the standard $\tau = 0.5$ threshold.

**Interpretation.**

- **ROC curve and AUC.** The ROC curve plots TPR against FPR as $\tau$ varies. An AUC close to 1 indicates strong discriminative ability; AUC = 0.5 corresponds to a random classifier. Our from-scratch AUC matches sklearn, confirming the implementation.

- **$F_\beta$-scores.** The parameter $\beta$ controls the precision-recall trade-off. $F_{0.5}$ penalises false positives more heavily (appropriate when false alarms are costly, e.g. spam filtering), $F_1$ treats precision and recall equally, and $F_2$ penalises false negatives more heavily ($\beta^2 = 4$ times the weight on recall vs precision).

- **Medical context.** In cancer screening, missing a malignant tumor (FN) is far more dangerous than an unnecessary follow-up biopsy (FP). Therefore $F_2$ is the most appropriate metric: its optimal threshold $\tau^*$ is lower than the $F_1$-optimal threshold, yielding higher recall at a modest cost in precision. The precision-recall curve visualises these operating points and their trade-offs.

***

## Section 5: Scikit-learn LogisticRegression and Coefficient Interpretation

**Theory recap.**  For each feature $j$, the **odds ratio** is $\text{OR}_j = e^{\beta_j}$. It represents the multiplicative change in the odds of being malignant when feature $j$ increases by one unit (one standard deviation, since features are standardized). An OR > 1 means the feature increases the probability of malignancy; OR < 1 means it decreases it.

### Question 5.1: Sklearn LogisticRegression

Fit `sklearn.linear_model.LogisticRegression` on all 30 features. Compare its coefficients and test predictions with the from-scratch implementation.

### Question 5.2: Odds Ratios and Feature Importance

Compute the odds ratios $\text{OR}_j = e^{\beta_j}$ for all 30 features using the sklearn model. Visualize them as a horizontal bar chart sorted by $|\beta_j|$. Identify the top 5 most predictive features and interpret them in the medical context.

**Interpretation.** The odds ratios quantify each feature's contribution to the diagnosis. Features related to tumor size and shape irregularity (e.g., `concave points_worst`, `radius_worst`) tend to have the largest positive coefficients, meaning they increase the odds of malignancy. This aligns with medical knowledge: larger, more irregular tumors are more likely to be malignant.

***

## Section 6: Feature Selection and Model Comparison

With 30 features, a natural question is: how many features do we actually need? We explore this by training logistic regression with increasing subsets of features, ranked by coefficient magnitude.

### Question 6.1: Accuracy vs. Number of Features

Train logistic regression with the top-$k$ features (ranked by $|\beta_j|$ from the full model) for $k = 1, 2, \ldots, 30$.  Plot test accuracy as a function of $k$ and identify the "elbow"---the minimum number of features that achieves near-optimal performance.

### Question 6.2: Model Comparison Summary

Compare four models---2-feature, top-5, top-10, and all-30---on the test set. Print a summary table of accuracy, precision, recall, and F1 for each.

**Interpretation.** The comparison reveals diminishing returns as we add more features. A handful of carefully selected features---those with the largest coefficients---capture the bulk of the predictive signal. This is a preview of **regularization** (next lesson), where we will formally penalize model complexity to achieve this feature selection automatically.

***

## Summary

| Component | From Scratch | Scikit-learn |
|-----------|-------------|-------------|
| Sigmoid | `sigmoid(z)` | Built into `LogisticRegression` |
| BCE Loss | `bce_loss(y, p)` | `log_loss` |
| Gradient | $(1/n)\mathbf{X}^T(\mathbf{p}-\mathbf{y})$ | L-BFGS optimizer |
| Training | `logistic_gd(...)` | `lr.fit(X, y)` |
| Prediction | `sigmoid(X @ beta)` | `lr.predict_proba(X)` |

**Key results from this workshop:**

1. The **sigmoid** maps linear predictions to probabilities; the **logit** is its inverse. Together they link the linear model to the Bernoulli likelihood.
2. The **BCE loss** is convex in $\boldsymbol{\beta}$, guaranteeing that gradient descent finds the global minimum. The gradient has the elegant form $\nabla J = \frac{1}{n}\mathbf{X}^T(\mathbf{p} - \mathbf{y})$.
3. The **threshold** $\tau$ controls the precision-recall trade-off. In medical diagnosis, lowering $\tau$ increases recall (fewer missed cancers) at the cost of precision.
4. **Odds ratios** $e^{\beta_j}$ provide interpretable feature importance: features related to tumor size and shape irregularity are the strongest predictors of malignancy.
5. **Feature selection** by coefficient magnitude shows diminishing returns, motivating the regularization techniques covered in the next lesson.

**Connection to GLMs:** Logistic regression is a special case of a **generalized linear model** (GLM) with a Bernoulli response distribution and the logit link function. The exponential family framework unifies logistic regression with linear regression (Gaussian + identity link) and other models (Poisson regression, etc.).